In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-cleaned-datasets/offer_metadata71.parquet
/kaggle/input/amex-cleaned-datasets/train_data_capped_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/add_trans.parquet
/kaggle/input/amex-cleaned-datasets/agg_trans_features.parquet
/kaggle/input/amex-cleaned-datasets/add_event_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/cleaned_test_dataa.parquet


In [24]:
import kagglehub

# Correct handle
path = kagglehub.dataset_download("aryansachannn/amex-cleaned-datasets")

print("✅ Dataset mounted at:", path)



✅ Dataset mounted at: /kaggle/input/amex-cleaned-datasets


In [25]:
import pandas as pd

df_meta = pd.read_parquet(f"{path}/offer_metadata71.parquet")


In [26]:
print(df_meta.columns.tolist())
df_meta.head()


['id3', 'id9', 'f375', 'f376', 'id10', 'f378', 'f374', 'id8', 'id12', 'id13', 'duration_days', 'offer_duration_category']


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id3,id9,f375,f376,id10,f378,f374,id8,id12,id13,duration_days,offer_duration_category
0,70687,FO5O,2,5.0,1,None,None,NaN,2018-01-01 00:00:00,2099-12-31 23:59:59,29949,permanent
1,900002526,UGE,2,100.0,1,None,None,NaN,2014-10-20 00:00:00,2099-12-31 23:59:59,31118,permanent
2,900002864,UTP,1,100.0,1,None,None,NaN,2016-07-19 00:00:00,2099-12-31 23:59:59,30480,permanent
3,19508,O,2,NaN,1,None,None,NaN,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,long_term
4,35903,O,2,NaN,1,None,None,NaN,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,long_term


In [27]:
import pandas as pd
import numpy as np

# Step 1: Load offer metadata
df_meta = pd.read_parquet("/kaggle/input/amex-cleaned-datasets/offer_metadata71.parquet")  # Adjust path if needed

# Step 2: Replace None-like strings with NaN
df_meta.replace(
    ['None', 'none', 'NaN', 'nan', '', 'NULL', 'null'],
    value=np.nan,
    inplace=True
)

# Step 3: Handle categorical columns
categorical_cols = ['id9', 'f374', 'offer_duration_category']

# Fill missing categorical entries with "missing"
for col in categorical_cols:
    if col in df_meta.columns:
        df_meta[col] = df_meta[col].fillna("missing")

# Step 4: One-hot encode select categorical columns
df_meta_encoded = pd.get_dummies(df_meta, columns=categorical_cols, drop_first=True)

# Step 5: Drop datetime columns if not needed
df_meta_encoded.drop(columns=["id12", "id13"], inplace=True, errors='ignore')

# Step 6: Save engineered features for merging
df_meta_encoded.to_parquet("offer_metadata71.parquet", index=False)
print("✅ Offer metadata features saved!")

# Optional: View sample
df_meta_encoded.head()


✅ Offer metadata features saved!


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id3,f375,f376,id10,f378,id8,duration_days,id9_$CPC,id9_$DEC,id9_$EC,...,f374_Women's & Children's Clothing,f374_Women's Accessory And Specialty Stores,f374_Women's Clothing Stores,f374_missing,offer_duration_category_long_term,offer_duration_category_permanent,offer_duration_category_quarterly,offer_duration_category_seasonal,offer_duration_category_short_term,offer_duration_category_weekly
0,70687,2,5.0,1,None,NaN,29949,False,False,False,...,False,False,False,True,False,True,False,False,False,False
1,900002526,2,100.0,1,None,NaN,31118,False,False,False,...,False,False,False,True,False,True,False,False,False,False
2,900002864,1,100.0,1,None,NaN,30480,False,False,False,...,False,False,False,True,False,True,False,False,False,False
3,19508,2,NaN,1,None,NaN,3499,False,False,False,...,False,False,False,True,True,False,False,False,False,False
4,35903,2,NaN,1,None,NaN,3499,False,False,False,...,False,False,False,True,True,False,False,False,False,False


In [28]:
df_offer['f378'].isna().sum()


2404

In [29]:
total_rows = len(df_offer)
missing_count = df_offer['f378'].isna().sum()
missing_percent = (missing_count / total_rows) * 100
print(f"Missing in f378: {missing_count} ({missing_percent:.2f}%)")


Missing in f378: 2404 (57.73%)


In [30]:
df_offer['f378'].dtype


dtype('O')

In [31]:
print(df_offer['f378'].unique())


[None 'gao$scbuyeectsamo$ioomporrarcwypfysalnypf1t7balst'
 'gao$scbuyeectsamo$ioomporrarcwypfysalhsf2t8balst' ...
 'g-a-buyeectsamo-ioomqpoyatotyecywh9dts-omut3tdtotst'
 'g-a-bpbuyeectsamo-ioomqpoyatotyecywh9dts-omut3tdtopst'
 'gao$scbuyeectsamo$ioompiato1r(ooasb6st']


In [33]:
df_offer['f378'] = df_offer['f378'].fillna('missing')
df_offer['f378'] = df_offer['f378'].replace('None', 'missing')


In [34]:
df_offer.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id3,id9,f375,f376,id10,f378,f374,id8,id12,id13,duration_days,offer_duration_category
0,70687,FO5O,2,5.0,1,missing,None,NaN,2018-01-01 00:00:00,2099-12-31 23:59:59,29949,permanent
1,900002526,UGE,2,100.0,1,missing,None,NaN,2014-10-20 00:00:00,2099-12-31 23:59:59,31118,permanent
2,900002864,UTP,1,100.0,1,missing,None,NaN,2016-07-19 00:00:00,2099-12-31 23:59:59,30480,permanent
3,19508,O,2,NaN,1,missing,None,NaN,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,long_term
4,35903,O,2,NaN,1,missing,None,NaN,2019-06-02 17:00:00,2028-12-31 16:59:59,3499,long_term


In [35]:
# Replace common string forms of missing values with np.nan
df_offer.replace(
    to_replace=['None', 'none', 'NaN', 'nan', '', 'NULL', 'null'],
    value=np.nan,
    inplace=True
)

# Check how many NaNs are now in each column
missing_counts = df_offer.isna().sum()
print("🔍 Missing values per column:\n", missing_counts[missing_counts > 0])


🔍 Missing values per column:
 f376    3186
f374     277
id8      277
dtype: int64


In [36]:
df_offer[['f374', 'id8']] = df_offer[['f374', 'id8']].fillna('missing')
df_offer['f376'] = df_offer['f376'].fillna(df_offer['f376'].median())


In [37]:
print("✅ Remaining NaNs in offer metadata:", df_offer.isna().sum().sum())


✅ Remaining NaNs in offer metadata: 0


In [39]:
import pandas as pd
import numpy as np

# Load the offer metadata file
df_offer = pd.read_parquet("/kaggle/input/amex-cleaned-datasets/offer_metadata71.parquet")  # Update path if needed

print("Original shape:", df_offer.shape)

# Replace 'None', 'nan', etc., with np.nan
df_offer.replace(
    to_replace=['None', 'none', 'NaN', 'nan', '', 'NULL', 'null'],
    value=np.nan,
    inplace=True
)

# Convert f376 to numeric if it's not already
df_offer['f376'] = pd.to_numeric(df_offer['f376'], errors='coerce')

# -------------------------------
# Grouping by offer ID (id3)
# -------------------------------

# 1. Numeric Aggregations
agg_offer = df_offer.groupby('id3').agg({
    'f376': ['mean', 'min', 'max', 'std', 'count'],
    'duration_days': 'mean',
})

# 2. Mode (most frequent category) for f378 and id8
agg_offer['f378_mode'] = df_offer.groupby('id3')['f378'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'missing')
agg_offer['id8_mode'] = df_offer.groupby('id3')['id8'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'missing')

# 3. One-hot column sums
onehot_cols = [col for col in df_offer.columns if col.startswith("f374_") or col.startswith("offer_duration_category_")]
onehot_sums = df_offer.groupby("id3")[onehot_cols].sum()

# -------------------------------
# Combine all features
# -------------------------------
agg_offer = agg_offer.reset_index()
if not onehot_sums.empty:
    agg_offer = agg_offer.merge(onehot_sums.reset_index(), on='id3', how='left')

# Final check
print("✅ Final engineered offer_metadata shape:", agg_offer.shape)
print(agg_offer.head())


Original shape: (4164, 12)
✅ Final engineered offer_metadata shape: (4164, 9)
    id3  f376                       duration_days  \
         mean   min   max std count          mean   
0   362   3.0   3.0   3.0 NaN     1          30.0   
1  1064   NaN   NaN   NaN NaN     0          91.0   
2  1068   NaN   NaN   NaN NaN     0          90.0   
3  1072  10.0  10.0  10.0 NaN     1          41.0   
4  1083   6.0   6.0   6.0 NaN     1          30.0   

                          f378_mode    id8_mode  
                                                 
0   g3baascbuyeectmpoakb1lo$bitscst  54990000.0  
1                           missing  58120000.0  
2                           missing  58120000.0  
3                           missing  59991300.0  
4  g6baascbuyeectmpiapjb1lo$bitscst  59440000.0  


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [40]:
# Flatten MultiIndex columns if they exist
agg_offer.columns = ['id3'] + [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
    for col in agg_offer.columns[1:]
]

# Final preview
print("✅ Cleaned column names:", agg_offer.columns.tolist())
agg_offer.head()


✅ Cleaned column names: ['id3', 'f376_mean', 'f376_min', 'f376_max', 'f376_std', 'f376_count', 'duration_days_mean', 'f378_mode_', 'id8_mode_']


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id3,f376_mean,f376_min,f376_max,f376_std,f376_count,duration_days_mean,f378_mode_,id8_mode_
0,362,3.0,3.0,3.0,NaN,1,30.0,g3baascbuyeectmpoakb1lo$bitscst,54990000.0
1,1064,NaN,NaN,NaN,NaN,0,91.0,missing,58120000.0
2,1068,NaN,NaN,NaN,NaN,0,90.0,missing,58120000.0
3,1072,10.0,10.0,10.0,NaN,1,41.0,missing,59991300.0
4,1083,6.0,6.0,6.0,NaN,1,30.0,g6baascbuyeectmpiapjb1lo$bitscst,59440000.0


In [41]:
print("Unique values in f378_mode_:", agg_offer["f378_mode_"].nunique())
print(agg_offer["f378_mode_"].value_counts().head(10))


Unique values in f378_mode_: 1252
f378_mode_
missing                                        2404
gao$scbuyeectsamo$ioompoasb1st                   17
gao$scbuyeectsamo$ioompoatb1st                   15
ga$scbuyeebctsamo$ioomepfuamioob1lo3sc(o$st      12
gao$scbuyeectsamo$ioompoamb1st                   11
gao$scbuyeectsamo$ioompoapb1st                   10
gao$scbuyeectsamo$ioompoacb1st                   10
gao$scbuyeectsamo$ioompoab1st                    10
g3baascbuyeectmpoakb1lo$bitscst                   9
gao$scbuyeectsamo$ioompoasb9st                    8
Name: count, dtype: int64


In [42]:
# Frequency encode f378_mode_
freq_map = agg_offer["f378_mode_"].value_counts().to_dict()
agg_offer["f378_mode_freq"] = agg_offer["f378_mode_"].map(freq_map)

# Optional: drop original column if not needed
agg_offer.drop(columns=["f378_mode_"], inplace=True)

# Check result
agg_offer[["id3", "f378_mode_freq"]].head()


,id3,f378_mode_freq
0,362,9
1,1064,2404
2,1068,2404
3,1072,2404
4,1083,1


In [43]:
agg_offer.drop(columns=["f378_mode_", "id8_mode_"], errors="ignore", inplace=True)


In [44]:
agg_offer.to_parquet("engineered_offer_metadata.parquet", index=False)
